In [1]:
# Importing Libraries
import numpy as np
from qiskit import *
from qiskit.utils import QuantumInstance

In [2]:
# According to the paper "T-count Optimized Design of Quantum Integer Multiplication" 
def control_add(n: int):
    
    ctrlr = QuantumRegister(1, 'ctrl')
    ar = QuantumRegister(n, 'a')
    br = QuantumRegister(n, 'b')
    anc = QuantumRegister(2, 'anc')
    ctrl_add_c = QuantumCircuit(ctrlr, ar, br, anc, name="Ctrl-Add")

    #Step 1:
    for i in range(1, n):
        ctrl_add_c.cx(ar[i], br[i])
    ctrl_add_c.barrier()

    #Step 2:
    ctrl_add_c.ccx(ctrlr[0], ar[n-1], anc[0])
    for i in range(n-2, 0, -1):
        ctrl_add_c.cx(ar[i],ar[i+1])
    ctrl_add_c.barrier()
    
    #Step 3:
    for i in range(n-1):
        ctrl_add_c.ccx(ar[i],br[i], ar[i+1])
    ctrl_add_c.barrier()
    
    #Step 4:
    ctrl_add_c.ccx(ar[n-1], br[n-1], anc[1])
    ctrl_add_c.ccx(ctrlr[0], anc[1], anc[0])
    ctrl_add_c.ccx(ar[n-1], br[n-1], anc[1])
    ctrl_add_c.ccx(ctrlr[0], ar[n-1], br[n-1])
    ctrl_add_c.barrier()
    
    #Step 5:
    for i in range(n-2,-1,-1):
        ctrl_add_c.ccx(br[i], ar[i], ar[i+1])
        ctrl_add_c.ccx(ctrlr[0], ar[i], br[i])
    ctrl_add_c.barrier()
    
    #Step 6:
    for i in range(1, n-1):
        ctrl_add_c.cx(ar[i], ar[i+1])
    ctrl_add_c.barrier()
    
    #Step 7:
    for i in range(1, n):
        ctrl_add_c.cx(ar[i], br[i])
    ctrl_add_c.barrier()
    
    print("Control Adder Circuit: ")
    print(ctrl_add_c)
    ctrl_add_c=ctrl_add_c.to_instruction()
    return ctrl_add_c

In [3]:
# According to the paper "T-count Optimized Design of Quantum Integer Multiplication" 
# Define the initial Toffolli Gate for multiply A0 with B0...Bn-1
def toffoli(n: int):
    ar = QuantumRegister(1, 'a')
    br = QuantumRegister(n, 'b')
    pr = QuantumRegister(n, 'p')
    toffoli_c = QuantumCircuit(ar, br, pr, name="Toffoli")
    for i in range(n):
        toffoli_c.ccx(ar[0], br[i], pr[i])
    print("Toffoli Circuit: ")
    print(toffoli_c)
    toffoli_c=toffoli_c.to_instruction()
    return toffoli_c

In [4]:
# According to the paper "T-count Optimized Design of Quantum Integer Multiplication" 
def multiplier(input1: int, input2: int):
    
    # Take the input and convert it to corresponding binary value
    a=bin(input1).replace("0b", "")
    b=bin(input2).replace("0b", "")
    
    # Take the maximum length of the from the binary value of inputs and keep the length of both same
    n=max(len(a), len(b))
    b=b.zfill(n)
    a=a.zfill(n)
    
    #Initilization of Quantum Registers 
    ar = QuantumRegister(n, 'a')
    br = QuantumRegister(n, 'b')
    pr = QuantumRegister(2*n, 'p')
    anc = QuantumRegister(1, 'anc')
    cr = ClassicalRegister(2*n, 'c')
    circ = QuantumCircuit(ar, br, pr, anc, cr)

    # Initilize the circuit based on the inputs
    for i in range(n):
        if a[n-i-1]=='1':
            circ.x(ar[i])
        if b[n-i-1]=='1':
            circ.x(br[i])
    
    # Apply Toffoli Gate Operation
    l=[ar[0]]
    for i in br:
        l.append(i)
    for i in pr[:n]:
        l.append(i)

    circ.append(toffoli(n), l)
    circ.barrier()
    
    # Apply the Control Adder operation 
    l=[]
    for i in br:
        l.append(i)

    for k in range(1,n-1):
        p=[ar[k]]
        p+=l
        for j in range(n+2):
            p.append(pr[k+j])
        circ.append(control_add(n),p)
        circ.barrier()
    
    if n>1:
        p=[ar[n-1]]+l
        for j in range(n+1):
            p.append(pr[n-1+j])
        p.append(anc[0])
        circ.append(control_add(n),p)
        circ.barrier()
    
    # Add Measurements to the circuit
    for i in range(len(pr)):
        circ.measure(pr[i], cr[i])
        
    print("Multiplier Circuit: ")
    print(circ)
    
    # Execute teh circuit on qasm_simulator
    backend = Aer.get_backend('qasm_simulator')
    instance = QuantumInstance(backend, shots=1000)
    
    # Get the reullts and take teh maximum value from the result 
    result = instance.execute(circ)
    counts = result.get_counts()
    res=max(counts, key=counts.get)
    return int(res,2)

In [5]:
ans=multiplier(5,6)
print("Answer: ", ans)

Toffoli Circuit: 
                    
  a: ──■────■────■──
       │    │    │  
b_0: ──■────┼────┼──
       │    │    │  
b_1: ──┼────■────┼──
       │    │    │  
b_2: ──┼────┼────■──
     ┌─┴─┐  │    │  
p_0: ┤ X ├──┼────┼──
     └───┘┌─┴─┐  │  
p_1: ─────┤ X ├──┼──
          └───┘┌─┴─┐
p_2: ──────────┤ X ├
               └───┘
Control Adder Circuit: 
                  ░            ░            ░                      ░           »
 ctrl: ───────────░───■────────░────────────░────────■─────────■───░────────■──»
                  ░   │        ░            ░        │         │   ░        │  »
  a_0: ───────────░───┼────────░───■────────░────────┼─────────┼───░────────┼──»
                  ░   │        ░ ┌─┴─┐      ░        │         │   ░        │  »
  a_1: ──■────────░───┼────■───░─┤ X ├──■───░────────┼─────────┼───░───■────■──»
         │        ░   │  ┌─┴─┐ ░ └─┬─┘┌─┴─┐ ░        │         │   ░ ┌─┴─┐  │  »
  a_2: ──┼────■───░───■──┤ X ├─░───┼──┤ X ├─░───■────┼────■────■───░─┤ X ├──